# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-low-spike" data-toc-modified-id="Prepare-low-spike-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare low-spike</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

Dependencies:

* OpenMS binaries in PATH (version 2.2.0)

## Setup

In [1]:
threads=6
run=batch11_maybe_final
mkdir ${run}

mkdir: cannot create directory ‘batch11_maybe_final’: File exists


: 1

In [2]:
sample_names_low="l1 l2 l3"
sample_names_high="h1 h2 h3"
samples="${sample_names_low} ${sample_names_high}"

In [3]:
echo -e "name\tbiorepgroup\ttechrepgroup\tcondition" > ${run}/design.tsv
echo -e "l1\t1\t1\tlow" >> ${run}/design.tsv
echo -e "l2\t2\t1\tlow" >> ${run}/design.tsv
echo -e "l3\t3\t1\tlow" >> ${run}/design.tsv
echo -e "h1\t1\t1\thigh" >> ${run}/design.tsv
echo -e "h2\t2\t1\thigh" >> ${run}/design.tsv
echo -e "h3\t3\t1\thigh" >> ${run}/design.tsv

cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition
l1	1	1	low
l2	2	1	low
l3	3	1	low
h1	1	1	high
h2	2	1	high
h3	3	1	high


## Prepare low-spike

In [4]:
lfqtk generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 0 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 50000 \
    --back_count 20 \
    --spike_count 5 \
    --spike_folds "1,1,1,8,8,8" \
    --out_base ${run} \
    --sample_names "l1.fa,l2.fa,l3.fa,h1.fa,h2.fa,h3.fa" \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l3.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h3.fa


# Simulate MS-data

Generate the OpenMS tool `MSSimulator`.

In [ ]:
MSSimulator -in ${run}/l1.fa -out_fm ${run}/l1.ground.featureXML

In [7]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML" \
    ::: ${samples}
#     -out_id ${run}/{}.ground.idXML \
#     -out ${run}/{}.mzML \
#     -out_pm ${run}/{}.centroided.mzML" \


Processing l1
Loading sequence data from batch11_maybe_final/l1.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 14:23:34: Digest Simulation ... started
2017/11/06, 14:23:34: RT Simulation ... started
2017/11/06, 14:23:35: Predicting RT ... done
2017/11/06, 14:23:35: RT prediction gave 'invalid' results for 736 peptide(s), making them unobservable.
2017/11/06, 14:23:35:   (List is too big to show)
2017/11/06, 14:23:35: Creating experiment with #501 scans ... done
2017/11/06, 14:23:35: Detectability Simulation ... started
2017/11/06, 14:23:35: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/06, 14:23:35: Simulating 840 features
Progress of 'Ionization':
-- done [took 4.45 s (CPU), 4.45 s (Wall)] -- 
2017/11/06, 14:23:40: #Peptides not ionized: 0
2017/11/06, 14:23:40: #Peptides outside mz range: 276
2017/11/06, 14:23:40: Raw MS1 Simulation ... started
2017/11/06, 14:23:40:   Simulating signal for 2330 features ...
Progress of '

-- done [took 3.02 s (CPU), 3.09 s (Wall)] -- 
2017/11/06, 14:23:46: Contaminants out-of-RT-range: 204 / 486
2017/11/06, 14:23:46: Contaminants out-of-MZ-range: 111 / 486
2017/11/06, 14:23:46: Compressed data to grid ... 4311088 --> 4062624 (94%)
2017/11/06, 14:23:46: Adding white noise to spectra ...
2017/11/06, 14:23:46: Adding detector noise to spectra ...
2017/11/06, 14:23:46: Detector noise was disabled.
2017/11/06, 14:23:46: Tandem MS Simulation ... disabled
2017/11/06, 14:23:47: Final number of simulated features: 2331
2017/11/06, 14:23:47: Simulation took 12.15531 seconds
2017/11/06, 14:23:47: Storing simulated features in: batch11_maybe_final/h1.ground.featureXML
2017/11/06, 14:23:47: MSSimulator took 12.35 s (wall), 12.20 s (CPU), 0.00 s (system), 12.20 s (user).
Processing h2
Loading sequence data from batch11_maybe_final/h2.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 14:23:34: Digest Simulation ... started
2017/11/06, 14:23:34: RT Simulation ... start

In [8]:
for xml in ${run}/*.featureXML; do 
    ProteinQuantifier \
        -in ${xml} \
        -peptide_out ${xml%.*}.csv
done


Processing summary - number of...
...features: 2331 used for quantification, 2331 total (0 no annotation, 0 ambiguous annotation)
...peptides: 837 quantified, 837 identified (considering best hits only)
ProteinQuantifier took 0.53 s (wall), 0.53 s (CPU), 0.00 s (system), 0.53 s (user).

Processing summary - number of...
...features: 2331 used for quantification, 2331 total (0 no annotation, 0 ambiguous annotation)
...peptides: 837 quantified, 837 identified (considering best hits only)
ProteinQuantifier took 0.55 s (wall), 0.54 s (CPU), 0.00 s (system), 0.54 s (user).

Processing summary - number of...
...features: 2331 used for quantification, 2331 total (0 no annotation, 0 ambiguous annotation)
...peptides: 837 quantified, 837 identified (considering best hits only)
ProteinQuantifier took 0.54 s (wall), 0.53 s (CPU), 0.00 s (system), 0.53 s (user).

Processing summary - number of...
...features: 2330 used for quantification, 2330 total (0 no annotation, 0 ambiguous annotation)
...pe

We receive identical peptide setup due to same protein being picked and cleaved (even if intensities varies).

In [12]:
ls

analyze_data.ipynb   batch5_pure        data
batch1               batch6_pure        full_quant.tsv
batch10_synced       batch7_no_align    prepare_data.ipynb
batch11_maybe_final  batch7_no_align_2  prepare_data_old_full_runthrough.ipynb
batch2               batch8             qc
batch3               batch9_synced      util_scripts
batch4_mzml          combine_quant.py


In [18]:
header="peptide\tprotein"
cut_string="1,2"

file_nbr=0
for sample in ${samples}; do
    ((file_nbr=file_nbr+1))
    header="${header}\t${sample}"    
    ((abund_col=file_nbr*5))
    cut_string="${cut_string},${abund_col}"    
done

echo ${header}
echo ${cut_string}

echo -e ${header} > ${run}/full_quant.tsv
paste ${run}/*.ground.csv | tail -n +2 | grep -v "^#" | tr -d "\"" | cut -f ${cut_string} \
    >> ${run}/full_quant.tsv

peptide\tprotein\tl1\tl2\tl3\th1\th2\th3
1,2,5,10,15,20,25,30


In [19]:
head ${run}/full_quant.tsv

peptide	protein	l1	l2	l3	h1	h2	h3
peptide	protein	abundance	abundance	abundance	abundance	abundance	abundance
FCR	sp|P0A7T7|RS18_ECOLI	151517200	165587200	144553300	150461100	159299600	150254600
LCR	sp|Q9JMR4|YUBK_ECOLI	44239800	52946600	46357600	47534300	49887400	46122300
FCQR	sp|P39357|YJHF_ECOLI	132788600	125155096	125492300	121695900	140206000	120414700
KFCR	sp|P0A7T7|RS18_ECOLI	69140400	73920540	66849810	70346130	73255450	68869210
AFVLR	sp|P0ADY1|PPID_ECOLI	134405600	128525900	127084400	124809200	125322200	140974800
GFVLR	sp|P33650|FEOB_ECOLI	124943800	128849200	142474900	138466500	131605800	130450700
GWPYA	sp|P58516|SPI3_SOLTU	317440992	319214016	317440992	37463300	40197300	40049600
MTLFR	sp|P77607|YAGL_ECOLI	121958700	135043000	131109200	124353500	132364100	117581296
